# HK Drayage Report – Prev vs Current Auto Comparison

- data/HK/drayage 폴더의 HK 리포트 중 최신 2개(전일/금일)를 자동 선택
- Master B/L No. + Container No. 기준으로 행 매칭
- ETA/ATA, LFD, PU APPT, DELIVERY DATE, DET FREE, EMPTY, Return, CNTR REMARK, Note 비교
- 변경/신규 행만 모아서 정렬 후 엑셀로 저장 (LFD → DELIVERY DATE 기준)

In [25]:
import pandas as pd
import numpy as np
from pathlib import Path
import re
from datetime import datetime

# ==========================
# 셀 1 — 파일 목록 & 비교 대상 자동 선택 (HK)
# ==========================

DATA_DIR = Path("data") / "HK" / "drayage"

files = list(DATA_DIR.glob("*.xlsx"))
if len(files) < 2:
    raise RuntimeError(f"HK drayage 폴더에 엑셀이 2개 이상 있어야 함: {DATA_DIR}")

def extract_date(path: Path):
    """
    파일명 안의 8자리 숫자 (MMDDYYYY) → datetime.date 로 변환
    예: 'HK 11202025.xlsx' → 11/20/2025
    """
    m = re.search(r"(\d{8})", path.stem)
    if not m:
        return None
    s = m.group(1)
    try:
        return datetime.strptime(s, "%m%d%Y").date()
    except ValueError:
        return None

files_with_dates = [(p, extract_date(p)) for p in files]
files_with_dates = [t for t in files_with_dates if t[1] is not None]

if len(files_with_dates) < 2:
    raise RuntimeError("날짜를 인식할 수 있는 HK 리포트가 2개 이상 필요함 (파일명에 MMDDYYYY 포함).")

files_sorted = sorted(files_with_dates, key=lambda x: x[1])
prev_file, prev_date = files_sorted[-2]
curr_file, curr_date = files_sorted[-1]

today = datetime.today().date()

print(f"- Today        : {today}")
print(f"- 이전 리포트     : {prev_date}  ({prev_file.name})")
print(f"- 최신 리포트     : {curr_date}  ({curr_file.name})")


- Today        : 2025-11-23
- 이전 리포트     : 2025-11-20  (HK 11202025.xlsx)
- 최신 리포트     : 2025-11-21  (HK 11212025.xlsx)


---
## Cell 2 — 전일/금일 HK 리포트 로드 + 컬럼 정리

In [26]:
# ==========================
# 셀 2 — 전일/오늘 리포트 로드 + 공통 컬럼 정리 (HK 포맷)
# ==========================

KEY = ["Master B/L No.", "Container No."]

DISPLAY_COLS = [
    "Master B/L No.",
    "Container No.",
    "ETA/ETD",
    "ATA",
    "Pick Up Ready Date",
    "Last Free Date",
    "P/U APPT DATE",
    "P/U APPT TIME",
    "DELIVERY DATE",
    "DET FREE DTE",
    "EMPTY NOTICE",
    "EMPTY PICK UP DATE",
    "Return Date",
    "CNTR REMARK",
    "Note",   # Unnamed: 11 → Note
]

DATE_COLS = [
    "ETA/ETD",
    "ATA",
    "Pick Up Ready Date",
    "Last Free Date",
    "P/U APPT DATE",
    "DELIVERY DATE",
    "DET FREE DTE",
    "EMPTY PICK UP DATE",
    "Return Date",
]

TIME_COLS = ["P/U APPT TIME"]

def load_hk_report(path: Path) -> pd.DataFrame:
    df = pd.read_excel(path)

    # 1) 컬럼명 공백 제거
    df.columns = df.columns.str.strip()

    # 2) DET FREE / Unnamed → 이름 통일
    rename_map = {}
    for c in df.columns:
        if c == "DET FREE DATE":
            rename_map[c] = "DET FREE DTE"
        if c.startswith("Unnamed"):
            rename_map[c] = "Note"
    if rename_map:
        df = df.rename(columns=rename_map)

    # 3) 사용 안 할 컬럼 제거
    for col in ["Pick Up Location", "SCAC"]:
        if col in df.columns:
            df = df.drop(columns=col)

    # 4) 필요한 컬럼이 없다면 추가 (NaN)
    for col in DISPLAY_COLS:
        if col not in df.columns:
            df[col] = pd.NA

    # 5) 컬럼 순서 통일
    df = df[DISPLAY_COLS]

    return df

df_prev = load_hk_report(prev_file)
df_curr = load_hk_report(curr_file)

# 날짜/시간 정규화
for df in (df_prev, df_curr):
    for col in DATE_COLS:
        df[col] = pd.to_datetime(df[col], errors="coerce").dt.date

    for col in TIME_COLS:
        s = pd.to_datetime(df[col], errors="coerce")
        df[col] = s.dt.strftime("%H:%M")
        df[col] = df[col].fillna("")

print("셀2 완료 — HK 리포트 로드 + 컬럼 정리/정규화 완료")


셀2 완료 — HK 리포트 로드 + 컬럼 정리/정규화 완료


--- 

## Cell 3 — 변경된 행/신규 행만 추출 (changed_table)

In [27]:
# ==========================
# 셀 3 — 변경 요약 + 변경 마스크 계산 (HK 최종본, MT → Empty 처리 포함)
# ==========================

# 0. MT → Empty 정규화 (EMPTY NOTICE, CNTR REMARK)
def normalize_mt_to_empty(val):
    if pd.isna(val):
        return val
    s = str(val).strip().upper()
    if s.startswith("MT"):
        return "Empty"
    return val

for col in ["EMPTY NOTICE", "CNTR REMARK"]:
    if col in df_prev.columns:
        df_prev[col] = df_prev[col].apply(normalize_mt_to_empty)
    if col in df_curr.columns:
        df_curr[col] = df_curr[col].apply(normalize_mt_to_empty)

# 1. 인덱스 설정
df_prev_k = df_prev.set_index(KEY)
df_curr_k = df_curr.set_index(KEY)

common_idx = df_prev_k.index.intersection(df_curr_k.index)
prev_c = df_prev_k.loc[common_idx]
curr_c = df_curr_k.loc[common_idx]

# 2. 공통 영역 비교 (NaN 동일 취급)
equal = (prev_c == curr_c) | (prev_c.isna() & curr_c.isna())
changed_mask = ~equal

# 컨테이너 기준 변경(하나라도 바뀌면 True)
row_changed = changed_mask.any(axis=1)
changed_keys = row_changed[row_changed].index
changed_cnt = int(row_changed.sum())

# 신규/삭제
added_idx   = df_curr_k.index.difference(df_prev_k.index)
deleted_idx = df_prev_k.index.difference(df_curr_k.index)

print("=== HK Drayage 리포트 변경 요약 ===")
print(f"- 신규 컨테이너 추가: {len(added_idx)} 건")
print(f"- 값이 변경된 컨테이너 행: {changed_cnt} 건")
print(f"- 삭제된 컨테이너: {len(deleted_idx)} 건")
print()
print("--- 날짜별 요약 ---")

# 3. 날짜/시간 컬럼 전부 요약
DATE_COLS = [
    "ETA/ETD",
    "ATA",
    "Pick Up Ready Date",
    "Last Free Date",
    "P/U APPT DATE",
    "P/U APPT TIME",
    "DELIVERY DATE",
    "DET FREE DTE",
    "EMPTY PICK UP DATE",
    "Return Date",
]

LABEL = {
    "ETA/ETD": "ETA/ETD",
    "ATA": "ATA",
    "Pick Up Ready Date": "픽업 레디 날짜",
    "Last Free Date": "LFD",
    "P/U APPT DATE": "픽업 날짜",
    "P/U APPT TIME": "픽업 시간",
    "DELIVERY DATE": "배송 예정일(Delivery)",
    "DET FREE DTE": "Detention Free",
    "EMPTY PICK UP DATE": "Empty 픽업 날짜",
    "Return Date": "리턴 날짜(Return)",
}

for col in DATE_COLS:
    if col not in df_curr_k.columns:
        continue

    label = LABEL.get(col, col)

    # Prev/Curr 전체 분포
    prev_s = df_prev_k[col]
    curr_s = df_curr_k[col]

    prev_cnt = prev_s.value_counts(dropna=True)
    curr_cnt = curr_s.value_counts(dropna=True)

    all_vals = prev_cnt.index.union(curr_cnt.index)

    # ✅ 날짜별 개수 변화의 절대값 합 = 총 변경 건수
    total_changed = 0
    for v in all_vals:
        p = int(prev_cnt.get(v, 0))
        c = int(curr_cnt.get(v, 0))
        if p == c:
            continue
        total_changed += abs(c - p)

    print(f"[{label}] 총 {total_changed} 건 변경")

    if total_changed == 0:
        print("  - 날짜별 분포 변화 없음")
        continue

    # 상세: 날짜별 분포 변화
    for v in all_vals:
        p = int(prev_cnt.get(v, 0))
        c = int(curr_cnt.get(v, 0))
        if p == c:
            continue

        try:
            v_str = v.strftime("%Y-%m-%d")
        except Exception:
            v_str = str(v)

        diff = c - p
        sign = "+" if diff >= 0 else ""
        print(f"  - {v_str}: {p} 건 → {c} 건 ({sign}{diff} 건)")

print()


=== HK Drayage 리포트 변경 요약 ===
- 신규 컨테이너 추가: 10 건
- 값이 변경된 컨테이너 행: 32 건
- 삭제된 컨테이너: 0 건

--- 날짜별 요약 ---
[ETA/ETD] 총 10 건 변경
  - 2025-11-23: 0 건 → 6 건 (+6 건)
  - 2025-11-24: 0 건 → 4 건 (+4 건)
[ATA] 총 10 건 변경
  - 2025-11-23: 0 건 → 6 건 (+6 건)
  - 2025-11-24: 0 건 → 4 건 (+4 건)
[픽업 레디 날짜] 총 4 건 변경
  - 2025-11-20: 4 건 → 5 건 (+1 건)
  - 2025-11-21: 0 건 → 3 건 (+3 건)
[LFD] 총 8 건 변경
  - 2025-11-18: 29 건 → 30 건 (+1 건)
  - 2025-11-19: 11 건 → 9 건 (-2 건)
  - 2025-11-20: 4 건 → 5 건 (+1 건)
  - 2025-11-25: 0 건 → 4 건 (+4 건)
[픽업 날짜] 총 10 건 변경
  - 2025-11-24: 0 건 → 6 건 (+6 건)
  - 2025-11-25: 0 건 → 4 건 (+4 건)
[픽업 시간] 총 20 건 변경
  - : 17 건 → 18 건 (+1 건)
  - 07:00: 55 건 → 58 건 (+3 건)
  - 08:00: 68 건 → 71 건 (+3 건)
  - 09:00: 59 건 → 64 건 (+5 건)
  - 10:00: 19 건 → 21 건 (+2 건)
  - 11:00: 21 건 → 20 건 (-1 건)
  - 14:00: 34 건 → 32 건 (-2 건)
  - 15:00: 20 건 → 19 건 (-1 건)
  - 17:00: 8 건 → 7 건 (-1 건)
  - 23:00: 13 건 → 14 건 (+1 건)
[배송 예정일(Delivery)] 총 6 건 변경
  - 2025-11-20: 0 건 → 1 건 (+1 건)
  - 2025-11-21: 0 건 → 5 건 (+5 건)
[Dete

## Cell 3 끝

---

## Final Report 

In [28]:
# ==========================
# 셀 4 — 변경된 컨테이너 상세 표 (HK 최종본)
# ==========================

import numpy as np
import re

# 1) 변경된 컨테이너 Curr 기준으로 추출
df_curr_k = df_curr.set_index(KEY)
df_prev_k = df_prev.set_index(KEY)

changed_curr = df_curr_k.loc[changed_keys]

print(f"변경된 컨테이너 수: {len(changed_curr)} 건\n")  # 반드시 32건

tbl = changed_curr.reset_index()

# ==========================
# 2) MT → Empty 치환
# ==========================
def normalize_mt_to_empty(val):
    if pd.isna(val):
        return ""
    s = str(val).strip().upper()
    if s.startswith("MT"):   # MT, MT 11/20, MT NOTICE 등 전부 Empty
        return "Empty"
    return val

for col in ["EMPTY NOTICE", "CNTR REMARK"]:
    if col in tbl.columns:
        tbl[col] = tbl[col].apply(normalize_mt_to_empty)


# ==========================
# 3) Note 제거 → CNTR REMARK → Remark
# ==========================
HK_COL_ORDER = [
    "Container No.",
    "Master B/L No.",
    "ETA/ETD",
    "ATA",
    "Pick Up Ready Date",
    "Last Free Date",
    "P/U APPT DATE",
    "P/U APPT TIME",
    "DELIVERY DATE",
    "DET FREE DTE",
    "EMPTY NOTICE",
    "EMPTY PICK UP DATE",
    "Return Date",
    "CNTR REMARK"
]

changed_table = tbl[HK_COL_ORDER].copy()
changed_table = changed_table.rename(columns={"CNTR REMARK": "Remark"})

# ==========================
# 4) 날짜 포맷 통일 (YYYY-MM-DD) + NaN → ""
# ==========================
DATE_COLS = [
    "ETA/ETD",
    "ATA",
    "Pick Up Ready Date",
    "Last Free Date",
    "P/U APPT DATE",
    "DELIVERY DATE",
    "DET FREE DTE",
    "EMPTY PICK UP DATE",
    "Return Date",
]

for col in DATE_COLS:
    dt = pd.to_datetime(changed_table[col], errors="coerce")
    changed_table[col] = dt.dt.strftime("%Y-%m-%d").fillna("")


# ==========================
# 5) 시간 포맷 통일 (경고 제거)
# ==========================
def parse_time_safe(x):
    if pd.isna(x):
        return ""
    s = str(x).strip()
    if re.match(r"^\d{1,2}:\d{2}$", s):   # 정확한 HH:MM만 허용
        return s
    return ""

changed_table["P/U APPT TIME"] = changed_table["P/U APPT TIME"].apply(parse_time_safe)


# ==========================
# 6) NaN → ""
# ==========================
changed_table = changed_table.fillna("")


# ==========================
# 7) 이전 값 있었는데 지금 빈칸이면 "(삭제)" 표시
# ==========================
display_to_src = {
    "ETA/ETD": "ETA/ETD",
    "ATA": "ATA",
    "Pick Up Ready Date": "Pick Up Ready Date",
    "Last Free Date": "Last Free Date",
    "P/U APPT DATE": "P/U APPT DATE",
    "P/U APPT TIME": "P/U APPT TIME",
    "DELIVERY DATE": "DELIVERY DATE",
    "DET FREE DTE": "DET FREE DTE",
    "EMPTY NOTICE": "EMPTY NOTICE",
    "EMPTY PICK UP DATE": "EMPTY PICK UP DATE",
    "Return Date": "Return Date",
    "Remark": "CNTR REMARK",
}

def non_empty(v):
    if pd.isna(v):
        return False
    if isinstance(v, str) and v.strip() == "":
        return False
    return True

for idx, row in changed_table.iterrows():
    key = (row["Master B/L No."], row["Container No."])
    prev_row = df_prev_k.loc[key]

    for disp_col, src_col in display_to_src.items():
        if disp_col not in changed_table.columns:
            continue
        curr_val = row[disp_col]
        prev_val = prev_row.get(src_col, None)

        # 이전엔 값 있었는데 현재는 빈칸 → (삭제)
        if (curr_val == "" or curr_val == None) and non_empty(prev_val):
            changed_table.at[idx, disp_col] = "(삭제)"


# ==========================
# 8) 정렬 (LFD → Delivery)
# ==========================
lfd_sort = pd.to_datetime(changed_table["Last Free Date"], errors="coerce")
dlv_sort = pd.to_datetime(changed_table["DELIVERY DATE"], errors="coerce")

changed_table["LFD_sort"] = lfd_sort.dt.date
changed_table["DLV_sort"] = dlv_sort.dt.date

changed_table = (
    changed_table
    .sort_values(by=["LFD_sort", "DLV_sort"], ascending=[True, True])
    .drop(columns=["LFD_sort", "DLV_sort"])
    .reset_index(drop=True)
)

changed_table.index = range(1, len(changed_table) + 1)


# ==========================
# 9) 하이라이트 마스크 생성
# ==========================
mask_for_style = pd.DataFrame(False, index=changed_table.index, columns=changed_table.columns)

# 원본 컬럼명 기준 비교
source_cols = [
    "ETA/ETD",
    "ATA",
    "Pick Up Ready Date",
    "Last Free Date",
    "P/U APPT DATE",
    "P/U APPT TIME",
    "DELIVERY DATE",
    "DET FREE DTE",
    "EMPTY NOTICE",
    "EMPTY PICK UP DATE",
    "Return Date",
    "CNTR REMARK",
]

source_cols = [c for c in source_cols if c in changed_mask.columns]

mask_src = changed_mask[source_cols].loc[changed_keys]

src_to_display = {
    "ETA/ETD": "ETA/ETD",
    "ATA": "ATA",
    "Pick Up Ready Date": "Pick Up Ready Date",
    "Last Free Date": "Last Free Date",
    "P/U APPT DATE": "P/U APPT DATE",
    "P/U APPT TIME": "P/U APPT TIME",
    "DELIVERY DATE": "DELIVERY DATE",
    "DET FREE DTE": "DET FREE DTE",
    "EMPTY NOTICE": "EMPTY NOTICE",
    "EMPTY PICK UP DATE": "EMPTY PICK UP DATE",
    "Return Date": "Return Date",
    "CNTR REMARK": "Remark",
}

for idx, row in changed_table.iterrows():
    key = (row["Master B/L No."], row["Container No."])
    if key not in mask_src.index:
        continue

    src_row = mask_src.loc[key]
    for src_col in source_cols:
        if bool(src_row[src_col]):
            mask_for_style.at[idx, src_to_display[src_col]] = True


# ==========================
# 10) "(삭제)" 셀은 무조건 변경 표시로 하이라이트
# ==========================
for idx in changed_table.index:
    for col in changed_table.columns:
        if changed_table.at[idx, col] == "(삭제)":
            mask_for_style.at[idx, col] = True


# ==========================
# 출력용 스타일 객체
# ==========================
def highlight(df):
    return np.where(mask_for_style.values, "background:#fff3b0;color:red;", "")

styled_table = (
    changed_table.style
    .set_table_styles([{"selector":"th","props":[("text-align","center")]}])
    .apply(highlight, axis=None)
)

styled_table


변경된 컨테이너 수: 32 건



,Container No.,Master B/L No.,ETA/ETD,ATA,Pick Up Ready Date,Last Free Date,P/U APPT DATE,P/U APPT TIME,DELIVERY DATE,DET FREE DTE,EMPTY NOTICE,EMPTY PICK UP DATE,Return Date,Remark
1,ZCSU7647719,ZIMUSEL200265382,2025-11-12,2025-11-12,2025-11-14,2025-11-18,2025-11-15,15:00,2025-11-15,2025-11-30,Empty,,2025-11-21,
2,ZCSU7978390,ZIMUSEL200265344,2025-11-12,2025-11-12,2025-11-14,2025-11-18,2025-11-17,18:00,2025-11-18,2025-12-02,Empty,,2025-11-21,
3,JXLU4503106,ZIMUSEL200265315,2025-11-12,2025-11-12,2025-11-14,2025-11-18,2025-11-18,08:00,2025-11-18,2025-12-03,Empty,,,
4,TCLU8497885,ZIMUSEL200265211,2025-11-12,2025-11-12,2025-11-14,2025-11-18,2025-11-18,09:00,2025-11-18,2025-12-03,Empty,,,
5,TIIU5787064,ZIMUSEL200265344,2025-11-12,2025-11-12,2025-11-14,2025-11-18,2025-11-18,14:00,2025-11-19,2025-12-03,Empty,,,
6,TCNU1046552,ZIMUSEL200265208,2025-11-12,2025-11-12,2025-11-14,2025-11-18,2025-11-18,14:00,2025-11-19,2025-12-03,Empty,,,
7,GAOU7679419,ZIMUSEL200265256,2025-11-12,2025-11-12,2025-11-14,2025-11-18,2025-11-20,23:00,2025-11-20,2025-12-05,,,,
8,ZCSU7360267,ZIMUSEL200265211,2025-11-12,2025-11-12,2025-11-15,2025-11-19,2025-11-19,17:00,2025-11-21,2025-12-04,,,,
9,JZPU8030885,ZIMUSEL200265209,2025-11-12,2025-11-12,2025-11-15,2025-11-19,2025-11-19,18:00,2025-11-21,2025-12-04,,,,
10,FSCU8150990,ZIMUSEL200265208,2025-11-12,2025-11-12,2025-11-15,2025-11-19,2025-11-19,18:00,2025-11-21,2025-12-04,,,,


Cell 4 Final Report End 

---

## Save

In [29]:
# ==========================
# 셀 5 — 엑셀 저장 (HK 최종본, 행/열 오프셋 수정)
# ==========================

import openpyxl
from openpyxl.styles import PatternFill, Font, Alignment
from openpyxl.utils import get_column_letter

# 저장 경로
out_path = f"output/hk_drayage_diff_{curr_date.strftime('%Y%m%d')}.xlsx"

# 1) 테이블 먼저 저장 (index=True)
changed_table.to_excel(out_path, index=True, sheet_name="HK-Changed")

# 2) 다시 로드해서 스타일 입히기
wb = openpyxl.load_workbook(out_path)
ws = wb.active   # HK-Changed 시트

# 스타일 정의
fill_yellow = PatternFill(start_color="FFF3B0", end_color="FFF3B0", fill_type="solid")
font_red = Font(color="FF0000")

# ==========================
# 헤더 가운데 정렬
# ==========================
for cell in ws[1]:
    cell.alignment = Alignment(horizontal="center", vertical="center")

# ==========================
# 변경된 셀 색칠
# ==========================
# changed_table.index: 1 ~ N
# 엑셀:
#   1행  : 인덱스 헤더(빈칸) + 컬럼 헤더
#   2행~ : 인덱스 값 + 데이터
# → DataFrame index i → 엑셀 row = i + 1
# → DataFrame 첫 컬럼 → 엑셀 col = 2 (col1은 인덱스)
for r_idx, row in changed_table.iterrows():
    excel_r = r_idx + 1  # 1 → 2행, 2 → 3행 ...

    for c_offset, col in enumerate(changed_table.columns, start=0):
        excel_c = 2 + c_offset  # 인덱스가 col1, 실제 데이터는 col2부터
        if mask_for_style.loc[r_idx, col]:
            cell = ws.cell(row=excel_r, column=excel_c)
            cell.fill = fill_yellow
            cell.font = font_red

# ==========================
# 열 너비 자동 조정
# ==========================
# col1(index) 포함해서 전체 넓이 조정
for col_idx in range(1, ws.max_column + 1):
    # 헤더 기준 최소 폭
    header_val = ws.cell(row=1, column=col_idx).value
    max_width = len(str(header_val)) if header_val is not None else 0

    # 데이터 기준 폭 확인
    for r in range(2, ws.max_row + 1):
        val = ws.cell(row=r, column=col_idx).value
        if val is not None:
            max_width = max(max_width, len(str(val)))

    ws.column_dimensions[get_column_letter(col_idx)].width = max_width + 2

wb.save(out_path)
print(f"엑셀 저장 완료: {out_path}")


엑셀 저장 완료: output/hk_drayage_diff_20251121.xlsx
